In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, time
from flask import Flask, request

app = Flask(__name__)

def in_between(now, start, end):
    if start <= end:
        return start <= now < end
    else: # over midnight e.g., 23:30-04:15
        return start <= now or now < end

def gkernel(x, b, x_data, y_data):
    xx = x - x_data
    K = (1/((np.sqrt(2*np.pi))))*np.exp(-0.5 *(xx/b)**2)
    while np.all(K==0):
        b = b*10
        K = (1/((np.sqrt(2*np.pi))))*np.exp(-0.5 *(xx/b)**2)
    Ksum = np.sum(K)
    weight = K/Ksum
    yk = sum(weight*y_data)
    xkyk = np.array([[x], [yk]])
    return xkyk


@app.route("/result", methods= ["POST","GET"])
def result():
    json = request.get_json()
    xesti = float(json["distance"]) * 1000
    
    # Read data_train
    train_set = np.genfromtxt('data_oct.csv', skip_header=1, delimiter=',')

    # Get speed data (kph) for regular, AM peak, PM peak
    speed_regular = train_set[:,0]
    speed_AMpeak = train_set[:,3]
    speed_PMpeak = train_set[:,5]

    speed_regular = speed_regular[~np.isnan(speed_regular)]
    speed_AMpeak = speed_AMpeak[~np.isnan(speed_AMpeak)]
    speed_PMpeak = speed_PMpeak[~np.isnan(speed_PMpeak)]

    # Get distance data (km) for regular, AM peak, PM peak
    # x: input parameter
    dist_regular = train_set[:,2]
    dist_AMpeak = train_set[:,4]
    dist_PMpeak = train_set[:,6]

    dist_regular = dist_regular[~np.isnan(dist_regular)]
    dist_AMpeak = dist_AMpeak[~np.isnan(dist_AMpeak)]
    dist_PMpeak = dist_PMpeak[~np.isnan(dist_PMpeak)]

    # Calculate travel time data (minutes) for regular, AM peak, PM peak
    # y: ground truth
    time_regular = dist_regular / speed_regular 
    time_AMpeak = dist_AMpeak / speed_AMpeak
    time_PMpeak = dist_PMpeak / speed_PMpeak
    
    now = datetime.now().time()

    if in_between(now, time(6), time(9)):
        period = 'AM peak' #Between 6 to 9AM
        b = 500
        x = dist_AMpeak
        y = time_AMpeak
    elif in_between(now, time(17), time(21)): 
        period = 'PM peak' #Between 5 to 9PM
        b = 500
        x = dist_PMpeak
        y = time_PMpeak
    else:
        period = 'Regular' #Outside AM and PM peak
        b = 100
        x = dist_regular
        y = time_regular
        
    xkyk = gkernel(xesti, b, x, y)

    out = {}
    
    out["Period"] = period
    out["ETA"] = xkyk[1][0]
    
    return (out)
    
if __name__ == '__main__':
    app.run(port=5000)